<a href="https://colab.research.google.com/github/harshitadd/CrossPriv/blob/master/CrossPriv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install syft==0.2.6 --quiet 
# !pip install pydicom --quiet

In [1]:
import os 
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker
import pydicom 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  dicom=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
    temp = pydicom.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(128,128))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  for PID in labels:
    for key in dcm_data:
      if(key[:-4]==PID[0]):
        l=[]
        for val in dcm_data[key]:
          l.append(scaler.fit_transform(val))
        l = np.reshape(l,(3,128,128))
        dicom.append(l)
        label.append(int(PID[5]))
  
  return dicom,label
    

In [3]:
dicom, label = makeNonFedData()

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 8
        self.test_batch_size = 4
        self.epochs = 25
        self.lr = 0.001
        self.log_interval = 10
args = Arguments()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [6]:
class data_maker(Dataset):
  def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 

        self.to_torchtensor()
        
  def __len__(self):
      return len(self.data)

  def to_torchtensor(self):      
      self.data=torch.from_numpy(self.data)
      self.labels=torch.from_numpy(self.targets)
  
  def __getitem__(self, idx):
      sample=self.data[idx]
      target=self.targets[idx]
      return sample,target

In [7]:
hook = sy.TorchHook(torch)
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [8]:
federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))

In [9]:
federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
test_data = data_maker(x_test,y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(8,8,padding=0)

        # Linear Layers 
        
        self.fc1 = nn.Linear(64*2*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        # print(x.shape)
        x = self.pool1(x)
        # print(x.shape)
        x = F.relu(self.conv2(x))
        # print(x.shape)
        x = self.pool2(x)
        # print(x.shape)
        x = x.view(-1,64*2*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
 
        return F.log_softmax(x,dim=1)

In [11]:
hospital_model = Net().double()
clinic_model = Net().double()
hospital_optimizer = optim.Adam(hospital_model.parameters(), lr=0.0001)
clinic_optimizer = optim.Adam(clinic_model.parameters(), lr=0.0001)
models = [hospital_model, clinic_model]
optimizers = [hospital_optimizer, clinic_optimizer]
model = Net().double()

In [12]:
compute_nodes = [hospital, clinic]

In [13]:
def train(args, model, federated_train_loader, optimizer, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(federated_train_loader): 
        model.send(data.location) 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(federated_train_loader), loss.item()))
    


In [14]:
def test(args, model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    #criterion = F.nll_loss()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, federated_train_loader, optimizer, epoch)
    test(args, model,  test_loader)



Train Epoch: 1 [0/304 (0%)]	Loss: 0.672762
Train Epoch: 1 [80/304 (26%)]	Loss: 0.669482
Train Epoch: 1 [160/304 (53%)]	Loss: 0.664675
Train Epoch: 1 [240/304 (79%)]	Loss: 0.661959

Test set: Average loss: 0.6497, Accuracy: 105/129 (81%)

Train Epoch: 2 [0/304 (0%)]	Loss: 0.657960
Train Epoch: 2 [80/304 (26%)]	Loss: 0.657128
Train Epoch: 2 [160/304 (53%)]	Loss: 0.652800
Train Epoch: 2 [240/304 (79%)]	Loss: 0.650527

Test set: Average loss: 0.6352, Accuracy: 105/129 (81%)

Train Epoch: 3 [0/304 (0%)]	Loss: 0.646872
Train Epoch: 3 [80/304 (26%)]	Loss: 0.646597
Train Epoch: 3 [160/304 (53%)]	Loss: 0.642013
Train Epoch: 3 [240/304 (79%)]	Loss: 0.639826

Test set: Average loss: 0.6211, Accuracy: 105/129 (81%)

Train Epoch: 4 [0/304 (0%)]	Loss: 0.636124
Train Epoch: 4 [80/304 (26%)]	Loss: 0.636909
Train Epoch: 4 [160/304 (53%)]	Loss: 0.632105
Train Epoch: 4 [240/304 (79%)]	Loss: 0.629956

Test set: Average loss: 0.6081, Accuracy: 105/129 (81%)

Train Epoch: 5 [0/304 (0%)]	Loss: 0.626157
Train